# MCP SQL Agent Demo

This notebook demonstrates how to use the Model Context Protocol (MCP) with LangChain to create an agent that can query multiple data sources:

- PostgreSQL (relational database)
- Snowflake (data warehouse)
- Grafana (metrics and dashboards)

The agent will be able to select the appropriate data source based on the user's question and use it to provide answers.

## Setup and Configuration

First, let's import the necessary libraries and set up our environment.

In [ ]:
# Import required libraries
import os
import sys
import json
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict, Any, Optional
from pathlib import Path

# Add project root to path for imports
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Import shared environment utilities
repo_root = Path.cwd().parent.parent.parent.parent
sys.path.append(str(repo_root))
from shared.utils.env import load_env

# Load environment variables
load_env()

In [ ]:
# Import LangChain components
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.tools import BaseTool, StructuredTool
from langchain.agents import AgentExecutor, create_openai_tools_agent

# Configure the LLM
llm = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY")
)

## Connect to MCP Servers

Now, let's set up connections to our MCP servers for each data source.

In [ ]:
from mcp.client import MCPClient

# Connect to PostgreSQL MCP server
postgres_mcp = MCPClient("http://mcp-postgres:8080")

# Connect to Snowflake MCP server
snowflake_mcp = MCPClient("http://mcp-snowflake:8080")

# Connect to Grafana MCP server
grafana_mcp = MCPClient("http://mcp-grafana:8080")

## Create Tools for LangChain Agent

Let's create tools for each data source that our agent can use.

In [ ]:
# PostgreSQL query tool
def query_postgres(query: str) -> str:
    """Execute a SQL query against PostgreSQL and return the results."""
    try:
        response = postgres_mcp.query(query)
        df = pd.DataFrame(response['rows'], columns=response['columns'])
        return df.to_markdown()
    except Exception as e:
        return f"Error querying PostgreSQL: {str(e)}"

postgres_tool = StructuredTool.from_function(
    func=query_postgres,
    name="query_postgres",
    description="Execute SQL queries against PostgreSQL database with e-commerce data (users, products, orders)"
)

# PostgreSQL schema info tool
def get_postgres_schema() -> str:
    """Get the schema information for PostgreSQL database."""
    try:
        schema_query = """
        SELECT 
            table_name, 
            column_name, 
            data_type, 
            is_nullable 
        FROM information_schema.columns 
        WHERE table_schema = 'public' 
        ORDER BY table_name, ordinal_position;
        """
        response = postgres_mcp.query(schema_query)
        df = pd.DataFrame(response['rows'], columns=response['columns'])
        return df.to_markdown()
    except Exception as e:
        return f"Error getting PostgreSQL schema: {str(e)}"

postgres_schema_tool = StructuredTool.from_function(
    func=get_postgres_schema,
    name="get_postgres_schema",
    description="Get schema information for PostgreSQL database tables and columns"
)

# Snowflake query tool
def query_snowflake(query: str) -> str:
    """Execute a SQL query against Snowflake and return the results."""
    try:
        response = snowflake_mcp.query(query)
        df = pd.DataFrame(response['rows'], columns=response['columns'])
        return df.to_markdown()
    except Exception as e:
        return f"Error querying Snowflake: {str(e)}"

snowflake_tool = StructuredTool.from_function(
    func=query_snowflake,
    name="query_snowflake",
    description="Execute SQL queries against Snowflake data warehouse"
)

# Grafana metrics query tool
def query_grafana_metrics(datasource: str, query: str, start: str, end: str, step: int = 60) -> str:
    """Query Grafana for metrics data."""
    try:
        response = grafana_mcp.query_metrics(datasource=datasource, query=query, start=start, end=end, step=step)
        return json.dumps(response, indent=2)
    except Exception as e:
        return f"Error querying Grafana metrics: {str(e)}"

grafana_metrics_tool = StructuredTool.from_function(
    func=query_grafana_metrics,
    name="query_grafana_metrics",
    description="Query Grafana for metrics data using PromQL or Flux queries"
)

# Grafana dashboards list tool
def get_grafana_dashboards() -> str:
    """Get a list of available Grafana dashboards."""
    try:
        response = grafana_mcp.get_dashboards()
        return json.dumps(response, indent=2)
    except Exception as e:
        return f"Error getting Grafana dashboards: {str(e)}"

grafana_dashboards_tool = StructuredTool.from_function(
    func=get_grafana_dashboards,
    name="get_grafana_dashboards",
    description="Get a list of available Grafana dashboards"
)

# Combine all tools
tools = [
    postgres_tool,
    postgres_schema_tool,
    snowflake_tool,
    grafana_metrics_tool,
    grafana_dashboards_tool
]

## Create the Agent

Now, let's create our SQL agent using LangChain and the tools we've defined.

In [ ]:
# Define the prompt template for our agent
prompt = ChatPromptTemplate.from_template("""
You are an expert SQL and data analytics agent that can query multiple data sources including PostgreSQL, Snowflake, and Grafana.

PostgreSQL contains e-commerce data with tables for users, products, orders, and order_items.

Always first check the schema of the database before running queries to understand the available tables and columns.

When analyzing data, provide insights and explanations about the results, not just raw data.

For time-series data from Grafana, consider adding visualizations where appropriate.

If you're unsure which data source to use, ask for clarification.

User's question: {input}
""")

# Create the agent
agent = create_openai_tools_agent(llm, tools, prompt)

# Create an agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## Test the Agent

Let's test our agent with some sample questions.

In [ ]:
# Test with a PostgreSQL query
response = agent_executor.invoke({"input": "What are the top 3 products by sales?"})
print(response["output"])

In [ ]:
# Test with a question about order statistics
response = agent_executor.invoke({"input": "What's the average order value and how many orders do we have per status?"})
print(response["output"])

## Try More Complex Queries

Now let's try some more complex scenarios that might require data from multiple sources.

In [ ]:
# Ask a question that might require multiple data sources
response = agent_executor.invoke({"input": "Can you analyze the relationship between server metrics and sales performance over the last week?"})
print(response["output"])

In [ ]:
# Interactive mode - ask your own questions
def ask_agent(question):
    response = agent_executor.invoke({"input": question})
    return response["output"]

# Example usage
question = "Which users have spent the most money, and what categories of products are they buying?"
answer = ask_agent(question)
print(answer)

## Conclusion

In this notebook, we've demonstrated how to use the Model Context Protocol (MCP) to create an agent that can query multiple data sources. The agent can analyze data from PostgreSQL, Snowflake, and Grafana, providing a unified interface for data exploration and analysis.

Key takeaways:

1. MCP provides a simple way to connect LLMs to various data sources
2. LangChain makes it easy to create agents that can use multiple tools
3. This approach allows for flexible, natural language interaction with your data